# 2023 CITS4012 Assignment
*Make sure you change the file name with your student id.*

# Readme
*If there is something to be noted for the marker, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style, please check the bottom of the this ipynb file*

In [4]:
import sys
sys.path.append('./src/QA')
from importlib import reload
from data_prep import DataPrep
reload(data_prep)
from data_prep import DataPrep

NameError: ignored

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

In [1]:
# only need to run these commands if using Colab
# Mount google drive and change directory to the git repo folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/CITS4012_2023
import sys
# colab seems to require an explicit path for this to work from mounted drive
sys.path.append('/content/gdrive/MyDrive/CITS4012_2023/src/QA')

Mounted at /content/gdrive
/content/gdrive/MyDrive/CITS4012_2023


In [ ]:
# when not using colab
import sys
sys.path.append('./src/QA')

In [6]:
text = "It's a nice day[]"

In [8]:
import re

In [29]:
re.sub('(?<=[^ ])(?=[.,!?()\'\"])|(?<=[.,!?()\'\"])(?=[^ ])', r' ', ).split(" ")

['The',
 'word',
 '"',
 'Computer',
 '"',
 'was',
 'removed',
 'from',
 'its',
 'name',
 'on',
 'January',
 '9',
 ',',
 '2007',
 ',',
 'reflecting',
 'its',
 'shifted',
 'focus',
 'towards',
 'consumer',
 'electronics',
 'after',
 'the',
 'introduction',
 'of',
 'the',
 'iPhone',
 '.']

In [2]:
from data_prep import DataPrep

In [3]:
df = DataPrep.parse_tsv('WikiQA-train.tsv')
question_id_list = df.loc[:, "QuestionID"]
question_id_list = list(set(question_id_list))
question_id_list.sort() # fix sorting or dont sort

question_id = {}
for i in df.index:
    row = df.loc[i]
    id = row["QuestionID"]
    if id not in question_id:
        question_id[id] = {"document": []}
        question_id[id]["question"] = row["Question"]
    
    question_id[id]["document"].append((row["Sentence"], row["Label"]))

In [4]:
toks = DataPrep.tokenize_question_and_doc(question_id["Q1"])
toks


(['how', 'are', 'glacier', 'caves', 'formed', '?'],
 [['A',
   'partly',
   'submerged',
   'glacier',
   'cave',
   'on',
   'Perito',
   'Moreno',
   'Glacier',
   '.'],
  ['The', 'ice', 'facade', 'is', 'approximately', '60', 'm', 'high'],
  ['Ice', 'formations', 'in', 'the', 'Titlis', 'glacier', 'cave'],
  ['A',
   'glacier',
   'cave',
   'is',
   'a',
   'cave',
   'formed',
   'within',
   'the',
   'ice',
   'of',
   'a',
   'glacier',
   '.'],
  ['Glacier',
   'caves',
   'are',
   'often',
   'called',
   'ice',
   'caves',
   ',',
   'but',
   'this',
   'term',
   'is',
   'properly',
   'used',
   'to',
   'describe',
   'bedrock',
   'caves',
   'that',
   'contain',
   'year-round',
   'ice',
   '.']],
 [['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA', 'OOA'],
  ['BOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA',
   'IOA

In [5]:
from word_embed import WordEmbed

[==================================================] 100.0% 128.1/128.1MB downloaded


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
embedded_document = WordEmbed.glove_embed(toks[1])

In [11]:
from word_embed import FeatExt

In [20]:
from nltk.tag import pos_tag


In [19]:
toks[0]

['how', 'are', 'glacier', 'caves', 'formed', '?']

In [21]:
# use for getting POS tags for question
pos_tag(toks[0])

[('how', 'WRB'),
 ('are', 'VBP'),
 ('glacier', 'JJ'),
 ('caves', 'NNS'),
 ('formed', 'VBN'),
 ('?', '.')]

In [22]:
# use for documents
pos_toks = FeatExt.get_pos_tags(toks[1])
print(pos_toks)
# need to convert into index

[['DT', 'RB', 'VBN', 'NN', 'NN', 'IN', 'NNP', 'NNP', 'NNP', '.'], ['DT', 'NN', 'NN', 'VBZ', 'RB', 'CD', 'NNS', 'JJ'], ['NNP', 'NNS', 'IN', 'DT', 'NNP', 'NN', 'NN'], ['DT', 'NN', 'NN', 'VBZ', 'DT', 'NN', 'VBN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', '.'], ['NNP', 'NNS', 'VBP', 'RB', 'VBN', 'NN', 'NNS', ',', 'CC', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'NN', 'NNS', 'WDT', 'VBP', 'JJ', 'NN', '.']]


# 2.QA Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

# 3.Model Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.1. Input Embedding Ablation Study

(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 



###3.2. Attention Ablation Study
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 

###3.3. Hyper Parameter Testing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title) 